# E9: Wallet Fusion (FAST VERSION - No PyG Install)

**Goal:** Combine pre-extracted E7-A3 embeddings with tabular features using XGBoost

**Key advantage:** No PyTorch Geometric installation needed! Just load pre-saved embeddings.

**Date:** November 11, 2025

## Step 1: Setup & Imports (No PyG!)

In [ ]:
import torch
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_curve, auc, roc_auc_score, f1_score, roc_curve
import xgboost as xgb

np.random.seed(42)
torch.manual_seed(42)

print("✓ Libraries loaded (no PyG needed!)")

## Step 2: Load Pre-extracted Embeddings

**NOTE:** Upload `e7_tx_embeddings.npy` and `e7_addr_embeddings.npy` to your Kaggle dataset.

If you don't have these, use the full version notebook to extract them once.

In [ ]:
# Load pre-extracted embeddings (MUCH FASTER!)
print("Loading pre-extracted embeddings...")

tx_embeddings = np.load('/kaggle/input/a3-embeddings/e7_tx_embeddings.npy')
addr_embeddings = np.load('/kaggle/input/a3-embeddings/e7_addr_embeddings.npy')

print(f"✓ Embeddings loaded:")
print(f"  Transaction: {tx_embeddings.shape}")
print(f"  Address: {addr_embeddings.shape}")

## Step 3: Load Labels & Splits

In [ ]:
print("Loading labels and splits...")

labels_df = pd.read_csv('/kaggle/input/elliptic-plus-plus/txs_classes.csv')
with open('/kaggle/input/elliptic-splits/splits.json') as f:
    splits = json.load(f)

# Convert to binary
labels = labels_df['class'].values
y = (labels == 1).astype(int)

# Create masks
train_mask = np.array(splits['train'])
val_mask = np.array(splits['val'])
test_mask = np.array(splits['test'])

print(f"✓ Labels loaded:")
print(f"  Fraud: {(y==1).sum()}, Licit: {(y==0).sum()}")
print(f"  Train: {train_mask.sum()}, Val: {val_mask.sum()}, Test: {test_mask.sum()}")

## Step 4: Load Tabular Features

In [ ]:
print("Loading transaction features...")

tx_features_df = pd.read_csv('/kaggle/input/elliptic-plus-plus/txs_features.csv')
tx_features = tx_features_df.iloc[:, 2:95].values  # Local features 1-93

print(f"✓ Features loaded: {tx_features.shape}")

## Step 5: Create Fusion Features

In [ ]:
print("Creating fusion features...")

# Normalize tabular features
scaler = StandardScaler()
tx_features_norm = scaler.fit_transform(tx_features[train_mask])
tx_features_norm_all = scaler.transform(tx_features)

# Create fusion: GNN embeddings (128) + Tabular (93) = 221 dims
tx_fusion = np.concatenate([tx_embeddings, tx_features_norm_all], axis=1)

print(f"✓ Fusion features: {tx_fusion.shape}")
print(f"  Embeddings (128) + Tabular (93) = 221 dims")

## Step 6: Train XGBoost Models

In [ ]:
# Calculate class weight
pos_weight = (y[train_mask] == 0).sum() / (y[train_mask] == 1).sum()

# XGBoost parameters
xgb_params = {
    'max_depth': 6,
    'learning_rate': 0.1,
    'n_estimators': 100,
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'scale_pos_weight': pos_weight,
    'random_state': 42,
    'tree_method': 'hist',
    'device': 'cuda' if torch.cuda.is_available() else 'cpu'
}

print(f"✓ XGBoost configured (device: {xgb_params['device']})")

In [ ]:
# Model 1: Tabular Only
print("\n" + "="*60)
print("Training Model 1: Tabular Only")
print("="*60)

model_tabular = xgb.XGBClassifier(**xgb_params)
model_tabular.fit(
    tx_features_norm_all[train_mask], 
    y[train_mask],
    eval_set=[(tx_features_norm_all[val_mask], y[val_mask])],
    verbose=10
)

pred_tabular = model_tabular.predict_proba(tx_features_norm_all[test_mask])[:, 1]
print("✓ Tabular model trained")

In [ ]:
# Model 2: Embeddings Only
print("\n" + "="*60)
print("Training Model 2: Embeddings Only")
print("="*60)

model_embeddings = xgb.XGBClassifier(**xgb_params)
model_embeddings.fit(
    tx_embeddings[train_mask], 
    y[train_mask],
    eval_set=[(tx_embeddings[val_mask], y[val_mask])],
    verbose=10
)

pred_embeddings = model_embeddings.predict_proba(tx_embeddings[test_mask])[:, 1]
print("✓ Embeddings model trained")

In [ ]:
# Model 3: Fusion
print("\n" + "="*60)
print("Training Model 3: FUSION (Embeddings + Tabular)")
print("="*60)

model_fusion = xgb.XGBClassifier(**xgb_params)
model_fusion.fit(
    tx_fusion[train_mask], 
    y[train_mask],
    eval_set=[(tx_fusion[val_mask], y[val_mask])],
    verbose=10
)

pred_fusion = model_fusion.predict_proba(tx_fusion[test_mask])[:, 1]
print("✓ Fusion model trained")

## Step 7: Evaluate & Compare

In [ ]:
def compute_metrics(y_true, y_pred_proba):
    """Compute PR-AUC, ROC-AUC, F1"""
    precision, recall, _ = precision_recall_curve(y_true, y_pred_proba)
    pr_auc = auc(recall, precision)
    
    roc_auc = roc_auc_score(y_true, y_pred_proba)
    
    fpr, tpr, thresholds = roc_curve(y_true, y_pred_proba)
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]
    y_pred_binary = (y_pred_proba >= optimal_threshold).astype(int)
    f1 = f1_score(y_true, y_pred_binary)
    
    return {
        'pr_auc': float(pr_auc),
        'roc_auc': float(roc_auc),
        'f1': float(f1),
        'threshold': float(optimal_threshold)
    }

# Compute metrics
y_test = y[test_mask]

results = {
    'tabular_only': compute_metrics(y_test, pred_tabular),
    'embeddings_only': compute_metrics(y_test, pred_embeddings),
    'fusion': compute_metrics(y_test, pred_fusion)
}

# Print results
print("\n" + "="*70)
print("E9 WALLET FUSION RESULTS")
print("="*70)

for model_name, metrics in results.items():
    print(f"\n{model_name.upper().replace('_', ' ')}:")
    print(f"  PR-AUC:   {metrics['pr_auc']:.4f}")
    print(f"  ROC-AUC:  {metrics['roc_auc']:.4f}")
    print(f"  F1:       {metrics['f1']:.4f}")

# Calculate improvements
fusion_vs_tabular = (results['fusion']['pr_auc'] - results['tabular_only']['pr_auc']) / results['tabular_only']['pr_auc'] * 100
fusion_vs_embeddings = (results['fusion']['pr_auc'] - results['embeddings_only']['pr_auc']) / results['embeddings_only']['pr_auc'] * 100

print(f"\n{'-'*70}")
print(f"FUSION IMPROVEMENT:")
print(f"  vs Tabular:    {fusion_vs_tabular:+.1f}%")
print(f"  vs Embeddings: {fusion_vs_embeddings:+.1f}%")
print("="*70)

# Save results
with open('e9_fusion_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("\n✓ Results saved")

## Step 8: Visualizations

In [ ]:
# Comparison bar chart
sns.set_style('whitegrid')
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

models = ['Tabular\nOnly', 'Embeddings\nOnly', 'Fusion']
metrics_names = ['PR-AUC', 'ROC-AUC', 'F1']
colors = ['#3498db', '#e74c3c', '#2ecc71']

for idx, metric_key in enumerate(['pr_auc', 'roc_auc', 'f1']):
    values = [
        results['tabular_only'][metric_key],
        results['embeddings_only'][metric_key],
        results['fusion'][metric_key]
    ]
    
    bars = axes[idx].bar(models, values, color=colors)
    axes[idx].set_ylabel(metrics_names[idx], fontsize=12)
    axes[idx].set_ylim([0, 1])
    axes[idx].set_title(f'{metrics_names[idx]} Comparison', fontsize=14, fontweight='bold')
    
    for bar, val in zip(bars, values):
        height = bar.get_height()
        axes[idx].text(bar.get_x() + bar.get_width()/2., height + 0.02,
                      f'{val:.4f}', ha='center', va='bottom', fontsize=10)
    
    best_idx = np.argmax(values)
    bars[best_idx].set_edgecolor('gold')
    bars[best_idx].set_linewidth(3)

plt.tight_layout()
plt.savefig('e9_fusion_comparison.png', dpi=300, bbox_inches='tight')
plt.show()
print("✓ Comparison chart saved")

In [ ]:
# PR and ROC curves
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# PR Curves
for name, pred, color, label in [
    ('tabular', pred_tabular, '#3498db', 'Tabular Only'),
    ('embeddings', pred_embeddings, '#e74c3c', 'Embeddings Only'),
    ('fusion', pred_fusion, '#2ecc71', 'Fusion')
]:
    precision, recall, _ = precision_recall_curve(y_test, pred)
    pr_auc = auc(recall, precision)
    axes[0].plot(recall, precision, color=color, lw=2.5, 
                label=f'{label} (PR-AUC={pr_auc:.4f})')

axes[0].set_xlabel('Recall', fontsize=12)
axes[0].set_ylabel('Precision', fontsize=12)
axes[0].set_title('Precision-Recall Curves', fontsize=14, fontweight='bold')
axes[0].legend(loc='best', fontsize=10)
axes[0].grid(True, alpha=0.3)

# ROC Curves
for name, pred, color, label in [
    ('tabular', pred_tabular, '#3498db', 'Tabular Only'),
    ('embeddings', pred_embeddings, '#e74c3c', 'Embeddings Only'),
    ('fusion', pred_fusion, '#2ecc71', 'Fusion')
]:
    fpr, tpr, _ = roc_curve(y_test, pred)
    roc_auc = auc(fpr, tpr)
    axes[1].plot(fpr, tpr, color=color, lw=2.5, 
                label=f'{label} (ROC-AUC={roc_auc:.4f})')

axes[1].plot([0, 1], [0, 1], 'k--', lw=1, alpha=0.3)
axes[1].set_xlabel('False Positive Rate', fontsize=12)
axes[1].set_ylabel('True Positive Rate', fontsize=12)
axes[1].set_title('ROC Curves', fontsize=14, fontweight='bold')
axes[1].legend(loc='best', fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('e9_fusion_curves.png', dpi=300, bbox_inches='tight')
plt.show()
print("✓ PR/ROC curves saved")

## Summary

In [ ]:
print("\n" + "="*70)
print("E9 WALLET FUSION COMPLETE (FAST VERSION)")
print("="*70)

print("\n⚡ Total time saved: ~10 minutes (no PyG installation!)")

print("\nKey Finding:")
if results['fusion']['pr_auc'] > max(results['tabular_only']['pr_auc'], results['embeddings_only']['pr_auc']):
    print("  ⭐ FUSION WINS: GNN + Tabular > Either alone")
    print(f"  ⭐ Best PR-AUC: {results['fusion']['pr_auc']:.4f}")
else:
    best_model = max(results, key=lambda k: results[k]['pr_auc'])
    print(f"  ⭐ {best_model.upper()} WINS: {results[best_model]['pr_auc']:.4f}")

print("\nOutput files:")
print("  ✓ e9_fusion_results.json")
print("  ✓ e9_fusion_comparison.png")
print("  ✓ e9_fusion_curves.png")
print("="*70)